In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import os

In [3]:
import boto3

In [4]:
import mlflow

In [5]:
from sklearn.pipeline import make_pipeline

In [6]:
aws_access_key_id = ''
aws_secret_access_key = ''
aws_region = ''

In [7]:
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_REGION'] = ''  # Replace with your region

In [8]:

mlflow.set_tracking_uri("http://ec2-13-60-13-236.eu-north-1.compute.amazonaws.com:5000/")

In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-flask-adocker-artifacts/1', creation_time=1720859614036, experiment_id='1', last_update_time=1720859614036, lifecycle_stage='active', name='green-taxi-duration-random-forest', tags={}>,
 <Experiment: artifact_location='s3://mlflow-flask-adocker-artifacts/0', creation_time=1720811197776, experiment_id='0', last_update_time=1720811197776, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
mlflow.set_experiment("green-taxi-duration-random-forest")

<Experiment: artifact_location='s3://mlflow-flask-adocker-artifacts/1', creation_time=1720859614036, experiment_id='1', last_update_time=1720859614036, lifecycle_stage='active', name='green-taxi-duration-random-forest', tags={}>

In [11]:
#session = boto3.Session(profile_name='myawsprofile')
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
# Example: Using the session to create a client
s3_client = session.client('s3')
s3_client

In [12]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [13]:
df_train = read_dataframe('C:/Users/aksha/Documents/Study/mlops-zero-to-one/02-Experiment Tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('C:/Users/aksha/Documents/Study/mlops-zero-to-one/02-Experiment Tracking/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [14]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="models")
    print(mlflow.get_artifact_uri())

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


C:\Users\aksha\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


s3://mlflow-flask-adocker-artifacts/1/845b2d40804f423bb03fdc81565f9598/artifacts


In [ ]:
from mlflow.tracking import MlflowClient


In [20]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '410d302e047e4a389cd0ad29458f327c'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [17]:
import mlflow
logged_model = 's3://mlflow-flask-adocker-artifacts/1/845b2d40804f423bb03fdc81565f9598/artifacts/models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.


In [23]:
loaded_model.predict()

DictVectorizer()